In [ ]:
#My Steps:
1. 

In [2]:
!pip install gridfs
!pip install PIL

ERROR: Could not find a version that satisfies the requirement gridfs (from versions: none)
ERROR: No matching distribution found for gridfs

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import face_recognition
import numpy as np
import pymongo
import gridfs
from PIL import Image
import cv2

In [5]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["attendance"]

In [2]:
import face_recognition
import pymongo
import numpy as np
from bson.binary import Binary

# Connect to the database
client = pymongo.MongoClient()
db = client["OriginalProject"]
collection = db["college_data"]

# Load the image and generate face encodings
image = face_recognition.load_image_file("../Sanvi.jpg")
database_face_locations = face_recognition.face_locations(image)
database_face_encodings = face_recognition.face_encodings(image, database_face_locations)[0]



# Convert the face encoding to a Binary object
binary_encoding = Binary(database_face_encodings.tobytes())

# Add the Binary object to the database along with the name of the person
person_name = "Sanvi Pokle"
img_path = "D:\DeepFace\Sanvi.jpg"
collection.insert_one({"name": person_name, "encoding": binary_encoding, "img_path" : img_path})


In [4]:
database_face_encodings



array([-0.1062582 ,  0.01046928,  0.10101628, -0.09719265, -0.14679955,
        0.054312  , -0.06077956, -0.01807227,  0.23480164, -0.16838737,
        0.17431012,  0.08587332, -0.14270861, -0.03420313, -0.04372324,
        0.1593143 , -0.09230657, -0.18386337, -0.07001125, -0.07140493,
       -0.08158876, -0.00247543,  0.0239342 ,  0.05928454, -0.16010764,
       -0.38391805, -0.03718141, -0.08544528,  0.01801637, -0.07297302,
       -0.09795616,  0.12162831, -0.18120483, -0.01918748,  0.04925338,
        0.15109912, -0.07639828,  0.017832  ,  0.20496508,  0.08287372,
       -0.26173458,  0.02981482,  0.06961746,  0.26504052,  0.1943963 ,
        0.0461435 ,  0.0445177 , -0.05558301,  0.19167025, -0.2777696 ,
        0.12479866,  0.16265145,  0.09946257,  0.04784302,  0.12317453,
       -0.09773673, -0.01530492,  0.10586181, -0.17097524,  0.0721087 ,
        0.03010796, -0.03994068, -0.07183007, -0.08000472,  0.22476122,
        0.18584119, -0.12028549, -0.14211537,  0.22671236, -0.15

The best match for this face is Unknown


In [7]:
#to extract faces from the image
import face_recognition
from PIL import Image
import pymongo
from deepface import DeepFace
import os
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# Load the input image
image = face_recognition.load_image_file("7.jpg")

# Find all the faces in the image
face_locations = face_recognition.face_locations(image)

# Loop through each face in the image
for i, face_location in enumerate(face_locations):
    # Extract the coordinates of the face
    top, right, bottom, left = face_location

    # Crop the face from the image
    face_image = image[top:bottom, left:right]
    
    
    # Save the face as a separate image
    pil_image = Image.fromarray(face_image)
    file_path = f"cropped_images/face_{i}_{timestamp}.jpg"

    pil_image.save(file_path)

    # Encode the face
    #face_encoding = face_recognition.face_encodings(face_image)[0]

    # Insert the face data into the database
    #faces_collection.insert_one({"image_path": file_path, "face_encoding": face_encoding.tolist(), "date": datetime.datetime.now()})

    

In [9]:
#set an appropriate tolerance level
import face_recognition
import pymongo
import numpy as np
from bson.binary import Binary
import os
import datetime
# Connect to the database
client = pymongo.MongoClient()
db = client["OriginalProject"]
collection = db["college_data"]
attendance_collection = db["student_attendance"]

# Load the known face encodings from the database
known_face_encodings = []
known_face_names = []
known_face_paths = []
for doc in collection.find():
    known_face_encodings.append(np.frombuffer(doc["encoding"], dtype=np.float64))
    known_face_names.append(doc["name"])
    known_face_paths.append(doc["img_path"])

# Loop through each cropped face image in the folder
for filename in os.listdir("cropped_images"):
    # Load the test image
    test_image = face_recognition.load_image_file(os.path.join("cropped_images", filename))

    # Find the face(s) in the test image
    test_face_locations = face_recognition.face_locations(test_image)
    test_face_encodings = face_recognition.face_encodings(test_image, test_face_locations)

    # Loop through each face encoding in the test image and find the best match in the known face encodings
    for test_face_encoding in test_face_encodings:
        # Find the index of the best match
        matches = face_recognition.compare_faces(known_face_encodings, test_face_encoding, tolerance=0.5)
        face_distances = face_recognition.face_distance(known_face_encodings, test_face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            # Retrieve the document with the matching encoding
            print(face_distances)
            doc = collection.find_one({"encoding": Binary(test_face_encoding.tobytes())})

            name = doc["name"] if doc is not None else "Unknown"
            cropped_img_path = known_face_paths[best_match_index]

            attendance_doc = {"name": known_face_names[best_match_index], "date": datetime.datetime.now(), "encoding": Binary(test_face_encoding.tobytes()), "image_path": os.path.join("cropped_images", filename)}
            attendance_collection.insert_one(attendance_doc)
           
            print(f"{known_face_names[best_match_index]} is present")
        else:
            print(f"No match found for {filename}")







[0.6576801  0.71037425 0.36758905]
Sanvi Pokle is present
[0.33540956 0.55842118 0.70447383]
Sahil Bane is present
[0.58190349 0.46093345 0.72581044]
Moeez Shaikh is present


In [13]:
for doc in attendance_collection.find():
    print(doc)

{'_id': ObjectId('641c73e29eaf899c2d1ecb6c'), 'name': 'Sahil Bane', 'date': datetime.datetime(2023, 3, 23, 21, 14, 34, 416000), 'encoding': b'\x00\x00\x00@Bi\xcc\xbf\x00\x00\x00 \xed\x10\x8e?\x00\x00\x00@F\xd0\x9b?\x00\x00\x00\xc0\xf6\xd2\x8d\xbf\x00\x00\x00\x00Y\xc3\xae\xbf\x00\x00\x00 p\xd3\xbe\xbf\x00\x00\x00@\x00]\xa2\xbf\x00\x00\x00@a\xfa\xb5\xbf\x00\x00\x00@\x89_\xc3?\x00\x00\x00\x00\xcb\xb9\xba\xbf\x00\x00\x00 =\x08\xcc?\x00\x00\x00@\xf3\xc0\x83?\x00\x00\x00\x80!\xa0\xc3\xbf\x00\x00\x00\x00\xf9y\xc0\xbf\x00\x00\x00\xc0\xde\xc3\xa4\xbf\x00\x00\x00 \xd4\xc0\xbe?\x00\x00\x00 \xf4\x9b\xb4\xbf\x00\x00\x00\x00\x88\x1e\xc2\xbf\x00\x00\x00`\xb5\x0e\xa0\xbf\x00\x00\x00\xc0\x08\xfe\xb5\xbf\x00\x00\x00\xe0Oe\xac?\x00\x00\x00\xe0\xb7\x10\x99\xbf\x00\x00\x00\xe0c{\x98\xbf\x00\x00\x00\x80^\xb9\x8f?\x00\x00\x00@GC\xc4\xbf\x00\x00\x00\x80\xc4j\xd7\xbf\x00\x00\x00\xa0\xda>\xb1\xbf\x00\x00\x00`\xbd\x82\xb9\xbf\x00\x00\x00\xe0\\\x10\xa3?\x00\x00\x00\xa0\xc0f\xbc\xbf\x00\x00\x00\x00\xcc\xafp?\x00\x

SyntaxError: invalid syntax. Perhaps you forgot a comma? (313051924.py, line 1)

In [22]:
from pymongo import MongoClient
from deepface import DeepFace
from PIL import Image
import numpy as np
from datetime import datetime
# Connect to the database
client = MongoClient()
db = client["OriginalProject"]
attendance_collection = db["student_attendance"]
.attendance_collection.delete_one({_id: new ObjectId('641c73e29eaf899c2d1ecb6c')};
today = datetime.today()
filter = {"date": {"$gte": datetime(today.year, today.month, today.day), "$lt": datetime(today.year, today.month, today.day + 1)}}
# Retrieve the image paths from the documents in the collection
image_paths = [doc["image_path"] for doc in attendance_collection.find()]
print(image_paths)
# Loop through each image path and analyze the emotions using DeepFace
for image_path in image_paths:
    # Open the image and display it
    image = Image.open(image_path)
    new_size = (512, 512)
    image = image.resize(new_size)
    image_np = np.array(image)
    image.show()
    
    # Analyze the emotions using DeepFace
    result = DeepFace.analyze(image_np, actions=['emotion'], enforce_detection=False)
    

    # Retrieve the dominant emotion
    emotions = result[0]['dominant_emotion']

    # Store the results in the database
    attendance_collection.update_one({"image_path": image_path},
                                     {"$set": {"emotions": emotions}})
    
    


SyntaxError: invalid syntax. Perhaps you forgot a comma? (803294653.py, line 10)

In [23]:
for image_path in image_paths:
    # Open the image and display it
    image = Image.open(image_path)
    new_size = (512, 512)
    image = image.resize(new_size)
    image_np = np.array(image)
    image.show()

'happy'

In [73]:
from pymongo import MongoClient
import datetime

# Connect to the database
client = MongoClient()
db = client["OriginalProject"]
attendance_collection = db["student_attendance"]

# Ask the user for the date
date_input = input("Enter the date (yyyy-mm-dd): ")
year, month, day = map(int, date_input.split("-"))
date = datetime.datetime(year=year, month=month, day=day)

# Filter the documents by the date
start_date = date.replace(hour=0, minute=0, second=0)
end_date = date.replace(hour=23, minute=59, second=59)
filter = {"date": {"$gte": start_date, "$lte": end_date}}
current_date_documents = attendance_collection.find(filter)

# Extract the name and emotion from the documents
names = []
emotions = []
print('Attendance and Emotions for the date:' ,date)
for document in current_date_documents:
    names.append(document["name"])
    emotions.append(document["emotions"])
    print(document['name'], ":" ,document['emotions'])


Enter the date (yyyy-mm-dd): 2023-03-23
Attendance and Emotions for the date: 2023-03-23 00:00:00
Sahil Bane : happy
Moeez Shaikh : happy


In [ ]:
from pymongo import MongoClient
from deepface import DeepFace
from PIL import Image
import numpy as np
import datetime

image = Image.open(image_path)
    new_size = (512, 512)
    image = image.resize(new_size)
    image_np = np.array(image)
    image.show()
